In [1]:
import os
import sys
import csv
import tqdm

from thefuzz import fuzz, process

# from fuzzyset import FuzzySet
# from cfuzzyset import cFuzzySet as FuzzySet

In [2]:
# Load the training data from the provided XML documents and parsed
# by the ../src/construct_normalization_training_data.py script.

fh = open('../data/train_xml_normalization_data.txt')
reader = csv.reader(fh)
header = next(reader)

training_map = dict()

for string, meddra_pt_id in reader:
    training_map[string.lower()] = meddra_pt_id
    
fh.close()

In [5]:
# Load all the strings from meddra 23.1 and map them to the preferred term id
fh = open('../data/meddra_pt_llt_map_omop_v23.1.csv')
reader = csv.reader(fh)
header = next(reader)

meddra_strings = dict()
# meddra_fuzzyset = FuzzySet()

for pt_concept_id, pt_concept_name, pt_meddra_id, llt_concept_id, llt_concept_name, llt_meddra_id in reader:
    
    meddra_strings[pt_concept_name.lower()] =  pt_meddra_id
    meddra_strings[llt_concept_name.lower()] = pt_meddra_id
    
#     meddra_fuzzyset.add(pt_concept_name.lower())
#     meddra_fuzzyset.add(llt_concept_name.lower())

fh.close()

len(meddra_strings)    

62571

In [6]:
terms_to_match = set()
for term in training_map.keys():
    if term in meddra_strings:
        # exact match, skip
        continue
    terms_to_match.add(term)

len(terms_to_match), len(training_map)

(1595, 2885)

In [7]:
correct = 0
incorrect = 0

choices = set(meddra_strings.keys())

for a in tqdm.tqdm(terms_to_match):
    
    fuzz_match = process.extractOne(a, choices)
    pred_meddra_pt_id = meddra_strings[fuzz_match[0]]

    if pred_meddra_pt_id == training_map[a]:
        correct += 1
    else:
        incorrect += 1

correct/(incorrect+correct)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1595/1595 [1:23:25<00:00,  3.14s/it]


0.3304075235109718

In [14]:
exact_matches = len(training_map)-len(terms_to_match)
exact_matches

1290

In [15]:
(correct+exact_matches)/(incorrect+correct+exact_matches)

0.6287694974003466

In [16]:
correct = 0
incorrect = 0

choices = set(meddra_strings.keys())

for a in tqdm.tqdm(terms_to_match):
    
    fuzz_match = process.extractOne(a, choices, scorer=fuzz.token_sort_ratio)
    pred_meddra_pt_id = meddra_strings[fuzz_match[0]]

    if pred_meddra_pt_id == training_map[a]:
        correct += 1
    else:
        incorrect += 1

correct/(incorrect+correct), (correct+exact_matches)/(incorrect+correct+exact_matches)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1595/1595 [18:37<00:00,  1.43it/s]


(0.4432601880877743, 0.6922010398613518)

In [ ]:
correct = 0
incorrect = 0

choices = set(meddra_strings.keys())

for a in tqdm.tqdm(terms_to_match):
    
    fuzz_match = process.extractOne(a, choices, scorer=fuzz.token_set_ratio)
    pred_meddra_pt_id = meddra_strings[fuzz_match[0]]

    if pred_meddra_pt_id == training_map[a]:
        correct += 1
    else:
        incorrect += 1

correct/(incorrect+correct), (correct+exact_matches)/(incorrect+correct+exact_matches)

  8%|██████████▌                                                                                                                              | 123/1595 [01:38<19:05,  1.28it/s]